In [1]:
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
import matplotlib.pyplot as plat
import warnings
warnings.filterwarnings("ignore")

import os
os.chdir("/Users/gilgu/OneDrive/Escritorio")

In [5]:
data = pd.read_csv("drugs.csv")
data.head(10)

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY
5,22,F,NORMAL,HIGH,8.607,drugX
6,49,F,NORMAL,HIGH,16.275,drugY
7,41,M,LOW,HIGH,11.037,drugC
8,60,M,NORMAL,HIGH,15.171,drugY
9,43,M,LOW,NORMAL,19.368,drugY


# # Convertir variables predictoras cualitativas a escala numérica.

In [6]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])
data['BP'] = le.fit_transform(data['BP'])
data['Cholesterol'] = le.fit_transform(data['Cholesterol'])

In [7]:
print(data)

     Age  Sex  BP  Cholesterol  Na_to_K   Drug
0     23    0   0            0   25.355  drugY
1     47    1   1            0   13.093  drugC
2     47    1   1            0   10.114  drugC
3     28    0   2            0    7.798  drugX
4     61    0   1            0   18.043  drugY
..   ...  ...  ..          ...      ...    ...
195   56    0   1            0   11.567  drugC
196   16    1   1            0   12.006  drugC
197   52    1   2            0    9.894  drugX
198   23    1   2            1   14.020  drugX
199   40    0   1            1   11.349  drugX

[200 rows x 6 columns]


# Dividir datos en conjunto de entrenamiento y prueba

In [8]:
X = data.drop('Drug', axis=1)
y = data['Drug']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def evaluate_decision_tree(criterion, max_depth):
    clf = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    report = classification_report(y_test, y_pred)
    return report

# Entrenar modelos con diferentes criterios y profundidades

In [9]:
# Evaluar con criterio Gini y Entropía, y varios niveles de profundidad

gini_report_depth_3 = evaluate_decision_tree('gini', 3)
gini_report_depth_5 = evaluate_decision_tree('gini', 5)
entropy_report_depth_3 = evaluate_decision_tree('entropy', 3)
entropy_report_depth_5 = evaluate_decision_tree('entropy', 5)

print("Reporte de clasificación para criterio Gini y profundidad 3:")
print(gini_report_depth_3)
print("\nReporte de clasificación para criterio Gini y profundidad 5:")
print(gini_report_depth_5)
print("\nReporte de clasificación para criterio de Entropía y profundidad 3:")
print(entropy_report_depth_3)
print("\nReporte de clasificación para criterio de Entropía y profundidad 5:")
print(entropy_report_depth_5)


Reporte de clasificación para criterio Gini y profundidad 3:
              precision    recall  f1-score   support

       drugA       1.00      1.00      1.00         6
       drugB       1.00      1.00      1.00         3
       drugC       0.00      0.00      0.00         5
       drugX       0.69      1.00      0.81        11
       drugY       1.00      1.00      1.00        15

    accuracy                           0.88        40
   macro avg       0.74      0.80      0.76        40
weighted avg       0.79      0.88      0.82        40


Reporte de clasificación para criterio Gini y profundidad 5:
              precision    recall  f1-score   support

       drugA       1.00      1.00      1.00         6
       drugB       1.00      1.00      1.00         3
       drugC       1.00      1.00      1.00         5
       drugX       1.00      1.00      1.00        11
       drugY       1.00      1.00      1.00        15

    accuracy                           1.00        40
   macro

In [10]:
# Recomendar medicamento para un paciente con datos específicos
# Primero, entrenamos un modelo con los mejores parámetros encontrados
best_clf = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=42)
best_clf.fit(X, y)

DecisionTreeClassifier(max_depth=5, random_state=42)

In [11]:
# Luego, predecimos el medicamento para el paciente con los datos proporcionados
patient_data = [[50, 0, 0, 1, 15.302]]  # Age:50, Sex: F, BP: HIGH, Cholesterol: NORMAL, Na_to_K: 15.302
predicted_drug = best_clf.predict(patient_data)[0]
print("\n El medicamento recomendado para el paciente es:", predicted_drug)


 El medicamento recomendado para el paciente es: drugY


In [12]:
from sklearn.tree import export_graphviz
import graphviz

# Entrenar el modelo con los mejores parámetros encontrados
best_clf = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=42)
best_clf.fit(X, y)

# Exportar el árbol de decisión a un archivo .dot
export_graphviz(best_clf, out_file='drug_tree.dot', feature_names=X.columns, class_names=best_clf.classes_, filled=True)

# Convertir el archivo .dot a un archivo PNG
with open('drug_tree.dot') as f:
    dot_graph = f.read()
graph = graphviz.Source(dot_graph)
graph.format = 'png'
graph.render('drug_tree', view=True)

'drug_tree.png'